# Top 40 Charts Functions

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
from glob import glob
from os import getcwd
from os.path import join
from fileUtils import getBasename, getDirname, getBaseFilename
from fsUtils import isFile, isDir, moveFile, removeFile, mkDir, setFile, moveFile
from searchUtils import findDirs, findExt
from webUtils import getHTML, getWebData
from timeUtils import getDateTime, isDate
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib

from discogs import discogs
from multiArtist import multiArtist
print("Python: {0}".format(sys.version))

from pandas import date_range
basedir = getcwd()
print("Basedir = {0}".format(basedir))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Basedir = /Users/tgadfort/Documents/code/charts
Notebook Last Run Initiated: 2020-01-09 18:08:04.218562


In [2]:
from datetime import date, timedelta

def all_sundays(year):
    # January 1st of the given year
       dt = date(year, 1, 1)
        # First Sunday of the given year       
       dt += timedelta(days = 6 - dt.weekday())  
       while dt.year == year:
          yield dt
          dt += timedelta(days = 7)

# Get Starter File

In [3]:
try:
    filename = glob(join(basedir, "data", "top40", "starter.html"))[0]
except:
    print("Could not find starter HTML file!")
fdata = getHTML(filename)

Could not find starter HTML file!


NameError: name 'filename' is not defined

In [7]:
chartData  = {}
dirname = None
baseURL = "http://top40-charts.com"

## Get Charts
def getCharts(fdata):
    charts = {}
    for iform,formdata in enumerate(fdata.findAll("form")):
        for isel,seldata in enumerate(formdata.findAll("select", {"name": "cid"})):
            for iop,opdata in enumerate(seldata.findAll("option")):
                    attrs  = opdata.attrs
                    value  = attrs['value']
                    charts[value] = opdata.text
    return charts


## Get Dates
def getDates(fdata):
    dates = []
    for iform,formdata in enumerate(fdata.findAll("form")):
        for isel,seldata in enumerate(formdata.findAll("select", {"name": "date"})):
            continue
            for iop,opdata in enumerate(seldata.findAll("option")):
                attrs  = opdata.attrs
                value  = attrs['value']
                dates.append(value)
    dates = sorted(list(set(dates)))
    return dates

charts = getCharts(fdata)

In [8]:
vals = getDates(fdata)

In [10]:
charts

{'36': 'Airplay World Official Top 100',
 '3': 'Argentina Top 20',
 '4': 'Australia Top 20',
 '6': 'Austria Top 20',
 '7': 'Belgium Top 20',
 '8': 'Brazil Top 20',
 '49': 'Bulgaria Top 20',
 '9': 'Canada Top 20',
 '37': 'Chile Top 20',
 '42': 'China Top 20',
 '32': 'Denmark Top 20',
 '5': 'Digital Sales Top 100',
 '31': 'Europe Official Top 100',
 '10': 'Finland Top 20',
 '11': 'France Top 20',
 '12': 'German Top 40',
 '2': 'Greece Top 20',
 '48': 'HeatSeekers Radio Tracks',
 '46': 'Hispanic America Top 40',
 '43': 'India Top 20',
 '15': 'Ireland Top 20',
 '18': 'Italy Top 20',
 '16': 'Japan Top 20',
 '24': 'Muchmusic Top 30',
 '13': 'Netherlands Top 20',
 '17': 'New Zealand Top 20',
 '19': 'Norway Top 20',
 '20': 'Portugal Top 20',
 '45': 'Russia Top 20',
 '21': 'Spain Top 20',
 '22': 'Sweden Top 20',
 '23': 'Switzerland Top 20',
 '14': 'Taiwan Top 10',
 '39': 'Top40-Charts.com Web Top 100',
 '25': 'UK Singles Top 40',
 '26': 'UK Top 20 Albums',
 '47': 'Ukraine Top 20',
 '28': 'USA Al

In [6]:
date_range(start='20-04-2019', end='25-05-2019', freq='W')

DatetimeIndex(['2019-04-21', '2019-04-28', '2019-05-05', '2019-05-12',
               '2019-05-19'],
              dtype='datetime64[ns]', freq='W-SUN')

In [50]:
for cid in charts.keys():
    url      = "https://top40-charts.com/chart.php?cid={0}"
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,}

    savedir  = join(basedir, "data", "top40", cname.replace("/", " "))
    mkDir(savedir)

    savename = setFile(savedir, "{0}.p".format(datename))
    if isFile(savename):
        continue

    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need

    print(idts,'/',len(dates),"\tSaving {0}".format(savename))
    saveJoblib(data=data, filename=savename, compress=True)
    sleep(3)

{'36': 'Airplay World Official Top 100',
 '3': 'Argentina Top 20',
 '4': 'Australia Top 20',
 '6': 'Austria Top 20',
 '7': 'Belgium Top 20',
 '8': 'Brazil Top 20',
 '49': 'Bulgaria Top 20',
 '9': 'Canada Top 20',
 '37': 'Chile Top 20',
 '42': 'China Top 20',
 '32': 'Denmark Top 20',
 '5': 'Digital Sales Top 100',
 '31': 'Europe Official Top 100',
 '10': 'Finland Top 20',
 '11': 'France Top 20',
 '12': 'German Top 40',
 '2': 'Greece Top 20',
 '48': 'HeatSeekers Radio Tracks',
 '46': 'Hispanic America Top 40',
 '43': 'India Top 20',
 '15': 'Ireland Top 20',
 '18': 'Italy Top 20',
 '16': 'Japan Top 20',
 '24': 'Muchmusic Top 30',
 '13': 'Netherlands Top 20',
 '17': 'New Zealand Top 20',
 '19': 'Norway Top 20',
 '20': 'Portugal Top 20',
 '45': 'Russia Top 20',
 '21': 'Spain Top 20',
 '22': 'Sweden Top 20',
 '23': 'Switzerland Top 20',
 '14': 'Taiwan Top 10',
 '39': 'Top40-Charts.com Web Top 100',
 '25': 'UK Singles Top 40',
 '26': 'UK Top 20 Albums',
 '47': 'Ukraine Top 20',
 '28': 'USA Al

# Download Chart Data

In [20]:
for cid, cname in charts.items():
    starterURL = "https://top40-charts.com/chart.php?cid={0}".format(cid)
    url = starterURL
    savedir  = join(basedir, "data", "top40")
    mkDir(savedir)
    savename = setFile(savedir, "{0}.p".format(cname.replace("/", " ")))
    
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,}
    
    if isFile(savename):
        continue

    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need

    print("URL ---> {0}".format(url))
    print(idts,'/',len(dates),"\tSaving {0}".format(savename))
    saveJoblib(data=data, filename=savename, compress=True)
    sleep(3)

URL ---> https://top40-charts.com/chart.php?cid=36
938 / 939 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100.p
  --> This file is 37.4kB.
URL ---> https://top40-charts.com/chart.php?cid=3
938 / 939 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Argentina Top 20.p
  --> This file is 26.9kB.
URL ---> https://top40-charts.com/chart.php?cid=4
938 / 939 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Australia Top 20.p
  --> This file is 27.7kB.
URL ---> https://top40-charts.com/chart.php?cid=6
938 / 939 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Austria Top 20.p
  --> This file is 27.8kB.
URL ---> https://top40-charts.com/chart.php?cid=7
938 / 939 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Belgium Top 20.p
  --> This file is 27.5kB.
URL ---> https://top40-charts.com/chart.php?cid=8
938 / 939 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Brazil Top 20.p
  --> This file is 26.5kB.
URL ---> https

In [31]:
chartsToGet = {}
for cid, cname in charts.items():
    chartsToGet[cid] = {}
    savename = setFile(savedir, "{0}.p".format(cname.replace("/", " ")))
    data = getHTML(savename)
    select = data.find("select", {"name": "date"})
    if select is None:
        raise ValueError("No dates!")
    for option in select.findAll("option"):
        attr  = option.attrs
        value = attr['value']
        url   = "https://top40-charts.com/chart.php?cid={0}&date={1}".format(cid, value)
        chartsToGet[cid][value] = url

# Download Charts

In [36]:
from timeUtils import getDateTime

In [39]:
getDateTime("2007-01-01").year

2007

In [53]:
from pathlib import Path
for cid, cidData in chartsToGet.items():
    cname   = charts[cid]
    savedir = join(basedir, "data", "top40", cname.replace("/", " "))

    for value, url in cidData.items():
        try:
            if getDateTime(value).year < 201:
                continue        
        except:
            continue
            
        if value is None:
            continue
        savename = setFile(savedir, "{0}.p".format(value))
        if savename is None:
            continue
            
        if isFile(savename):
            print("Touching {0}".format(savename))
            Path(savename).touch()
        else:
            user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
            headers={'User-Agent':user_agent,}
    
            request=urllib.request.Request(url,None,headers) #The assembled request
            response = urllib.request.urlopen(request)
            data = response.read() # The data u need

            print("URL ---> {0}".format(url))
            print(idts,'/',len(dates),"\tSaving {0}".format(savename))
            saveJoblib(data=data, filename=savename, compress=True)
            sleep(2)

Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-01-07.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-01-14.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-01-21.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-02-02.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-02-09.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-02-16.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-02-23.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-03-02.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-03-09.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official T

Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2004-09-11.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2004-09-18.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2004-09-25.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2004-10-02.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2004-10-09.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2004-10-16.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2004-10-23.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2004-10-30.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2004-11-06.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official T

Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2007-06-16.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2007-06-23.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2008-01-19.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2008-01-26.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2008-02-02.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2008-02-09.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2008-02-16.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2008-04-26.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2008-05-03.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official T

Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2012-07-12.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2012-07-19.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2012-07-26.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2012-08-02.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2012-08-09.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2012-08-16.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2012-08-23.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2012-08-30.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2012-09-06.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official T

Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2018-08-18.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2018-08-25.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2018-09-01.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2018-09-08.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2018-09-15.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2018-09-22.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2018-09-29.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2018-10-06.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2018-10-13.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official T

...

Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2007-01-06.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2007-01-13.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2007-01-20.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2007-01-27.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2007-02-03.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2007-02-10.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2007-02-17.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2007-02-24.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2007-03-03.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2007-03-10.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2007-03-17.p
Touching /

Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2009-04-25.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2009-05-02.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2009-05-09.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2009-05-16.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2009-05-23.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2009-05-30.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2009-06-06.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2009-06-13.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2009-06-20.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2009-06-27.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2009-07-04.p
Touching /

Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2012-08-04.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2012-08-11.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2012-08-18.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2012-08-25.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2012-09-01.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2012-09-08.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2012-09-15.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2012-09-22.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2012-09-29.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2012-10-06.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2012-10-13.p
Touching /

Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2016-04-16.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2016-04-23.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2016-04-30.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2016-05-07.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2016-05-14.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2016-05-21.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2016-05-28.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2016-06-04.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2016-06-11.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2016-06-18.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2016-06-25.p
Touching /

Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2019-08-24.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2019-08-31.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2019-09-07.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2019-09-14.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2019-09-21.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2019-09-28.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2019-10-05.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2019-10-12.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2019-10-19.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2019-10-26.p
Touching /Users/tgadfort/Documents/code/charts/data/top40/USA Singles Top 40/2019-11-02.p
Touching /

# Parse Charts

In [54]:
def getChartData(chartData, debug=False):
    results = []
    pos = 1
    
    debVars = None
    
    for it,table in enumerate(chartData.findAll("table")):
        ths = table.findAll("th")
        trs = table.findAll("tr")
        attrs = table.attrs
        if debug:
            print(it,len(ths),len(trs),attrs)

        if attrs == {'cellpadding': '0', 'cellspacing': '0', 'borer': '0'}:
            if len(trs) == 1:
                tds = trs[0].findAll("td")
                if len(tds) == 3:
                    refs = tds[2].findAll("a")
                    if len(refs) == 2:
                        album  = refs[0].text
                        artist = refs[1].text
                        results.append({"Artist": artist, "Album": album})
                        if debug:
                            print(pos,'\t',artist,'\t',album)
                        pos += 1

    return results

In [60]:
for cid, cidData in chartsToGet.items():
    chartResults = {}
    cname   = charts[cid]
    print(cname)
    if cname == "World Adult Top 20 Singles":
        continue
    if cname.startswith("W"):
        pass
    else:
        continue
    savedir = join(basedir, "data", "top40", cname.replace("/", " "))

    for value, url in cidData.items():
        savename = setFile(savedir, "{0}.p".format(value))
        data     = getHTML(savename)
        results  = getChartData(data)
        chartResults[value] = results
        if len(results) == 0:
            print('\t',cname,'\t\t',value,'\t',len(results)," <<-------")
        else:
            print('\t',cname,'\t\t',value,'\t',len(results))
            
    savedir = join(basedir, "results", "top40")
    savename = setFile(savedir, "{0}.p".format(cname.replace("/", " ")))
    saveJoblib(data=chartResults, filename=savename, compress=True)
    #savename = setFile("~/Dropbox/charts", "{0}.p".format(cname.replace("/", " ")))
    #saveJoblib(data=chartResults[cid], filename=savename, compress=True)

Airplay World Official Top 100
Argentina Top 20
Australia Top 20
Austria Top 20
Belgium Top 20
Brazil Top 20
Bulgaria Top 20
Canada Top 20
Chile Top 20
China Top 20
Denmark Top 20
Digital Sales Top 100
Europe Official Top 100
Finland Top 20
France Top 20
German Top 40
Greece Top 20
HeatSeekers Radio Tracks
Hispanic America Top 40
India Top 20
Ireland Top 20
Italy Top 20
Japan Top 20
Muchmusic Top 30
Netherlands Top 20
New Zealand Top 20
Norway Top 20
Portugal Top 20
Russia Top 20
Spain Top 20
Sweden Top 20
Switzerland Top 20
Taiwan Top 10
Top40-Charts.com Web Top 100
UK Singles Top 40
UK Top 20 Albums
Ukraine Top 20
USA Albums
USA Singles Top 40
World Adult Top 20 Singles
World Country Top 20 Singles


ValueError: Could not parse html data!

# Aggregate Charts

In [4]:
from searchUtils import findExt
savedir = join(basedir, "results", "top40")
files = findExt(savedir, ".p")

In [15]:
from fileUtils import getBaseFilename
results = {}
for ifile in files:
    chartData = getFile(ifile)
    cname     = getBaseFilename(ifile)
    print(cname,'\t',end="")
    for date, values in chartData.items():
        for i,item in enumerate(values):
            artist = item["Artist"]
            album  = item["Album"]
            if results.get(artist) is None:
                results[artist] = {"Songs": {}, "Albums": {}}
            if cname.endswith("Albums"):
                key = "Albums"
            else:
                key = "Songs"
            if results[artist][key].get(album) is None:
                results[artist][key][album] = {}
            if results[artist][key][album].get(cname) is None:
                results[artist][key][album][cname] = {}
            results[artist][key][album][cname][date] = i
    print(len(results))

Sweden Top 20 	1931
New Zealand Top 20 	2815
UK Top 20 Albums 	3246
Russia Top 20 	3651
USA Singles Top 40 	4501
Switzerland Top 20 	5048
India Top 20 	5135
Canada Top 20 	5451
France Top 20 	6471
Norway Top 20 	6894
Austria Top 20 	7544
Chile Top 20 	8050
Belgium Top 20 	8499
Portugal Top 20 	8791
Hispanic America Top 40 	8861
Airplay World Official Top 100 	10472
UK Singles Top 40 	10835
China Top 20 	11006
Muchmusic Top 30 	11138
USA Albums 	11964
Brazil Top 20 	12377
Australia Top 20 	12698
Ukraine Top 20 	13173
Top40-Charts.com Web Top 100 	13449
Greece Top 20 	13908
Italy Top 20 	14077
Spain Top 20 	14495
HeatSeekers Radio Tracks 	14641
Argentina Top 20 	14835
Europe Official Top 100 	15786
German Top 40 	15924
Denmark Top 20 	16169
Finland Top 20 	16757
Digital Sales Top 100 	16798
Japan Top 20 	17312
Bulgaria Top 20 	17467
Taiwan Top 10 	17563
Ireland Top 20 	17711
Netherlands Top 20 	18211


In [21]:
from collections import Counter
slimResults = Counter()
for artist, artistData in results.items():
    for key, keyData in artistData.items():
        for album, albumData in keyData.items():
            slimResults[artist] += sum({k: len(v) for k,v in albumData.items()}.values())

In [26]:
saveFile(ifile="chartCounter.p", idata=slimResults, debug=True)

Saving data to chartCounter.p
  --> This file is 244.4kB.
Saved data to chartCounter.p
  --> This file is 244.4kB.


In [18]:
for artist, artistData in results.items():
    print(artist)
    print("    Songs:")
    for album, albumData in artistData["Songs"].items():
        print('\t',album,'\t',len(albumData))
        for chart, chartData in albumData.items():
            pass
            #print('\t\t',chart,chartData)
    print("    Albums:")
    for album, albumData in artistData["Albums"].items():
        print('\t',album,'\t',len(albumData))
        for chart, chartData in albumData.items():
            pass
            #print('\t\t',chart,chartData)
    break

Shaggy
    Songs:
	 Angel 	 16
	 Luv Me, Luv Me 	 11
	 Hope 	 6
	 Hey Sexy Lady 	 16
	 Feel The Rush 	 6
	 Dance & Shout 	 1
	 It Wasn't Me 	 5
	 Strength Of A Woman 	 6
	 Fly High 	 6
	 Freaky Girl 	 1
    Albums:
	 Hot Shot 	 1
	 Mr Lover Lover - The Best Of 	 1


In [14]:
daytype = {"USA Albums": ["SAT", "%Y-%m-%d"],
           "USA Singles Top 40": ["SAT", "%Y-%m-%d"]}
daytype = {"UK Singles Top 40": ["SAT", "%Y-%m-%d"]}
daytype = {"Top40-Charts.com Web Top 100": ["SAT", "%Y-%m-%d"]}
daytype = {"Airplay World Official Top 100": ["SAT", "%Y-%m-%d"]}

#daytype = {"USA Singles Top 40": ["SAT", "%Y-%m-%d"]}

from pandas import Timestamp
def getDates(daytype, cname):
    dates = []
    if cname == "USA Singles Top 40":
        dates1 = date_range(start='1997-07-07', end="2000-04-01", freq="W-MON").strftime("%Y-%m-%d")
        dates2 = date_range(start="2000-04-01", end=Timestamp.today(), freq="W-SAT").strftime("%Y-%m-%d")
        dates = list(dates1) + list(dates2)
    if cname == "UK Singles Top 40":
        dates1 = date_range(start="2002-12-08", end='2011-11-12', freq="W-SUN").strftime("%Y-%m-%d")
        dates2 = date_range(start="2011-11-12", end=Timestamp.today(), freq="W-SAT").strftime("%Y-%m-%d")
        dates = list(dates1) + list(dates2)        
    if cname == "Top40-Charts.com Web Top 100":
        dates1 = date_range(start="2002-10-14", end='2005-12-26', freq="W-MON").strftime("%Y-%m-%d")
        dates2 = date_range(start="2009-02-28", end=Timestamp.today(), freq="W-SAT").strftime("%Y-%m-%d")
        dates = list(dates1) + list(dates2)
    if cname == "Airplay World Official Top 100":
        dates1 = date_range(start="2002-01-07", end='2002-01-21', freq="W-MON").strftime("%Y-%m-%d")
        dates2 = date_range(start="2002-02-02", end=Timestamp.today(), freq="W-SAT").strftime("%Y-%m-%d")
        dates = list(dates1) + list(dates2)
    return dates

In [133]:
def showDates(year, day):
    return date_range(start=str(year), end=str(int(year)+1), freq='W-{0}'.format(day))
    
showDates(2011, 'SUN')

DatetimeIndex(['2011-01-02', '2011-01-09', '2011-01-16', '2011-01-23',
               '2011-01-30', '2011-02-06', '2011-02-13', '2011-02-20',
               '2011-02-27', '2011-03-06', '2011-03-13', '2011-03-20',
               '2011-03-27', '2011-04-03', '2011-04-10', '2011-04-17',
               '2011-04-24', '2011-05-01', '2011-05-08', '2011-05-15',
               '2011-05-22', '2011-05-29', '2011-06-05', '2011-06-12',
               '2011-06-19', '2011-06-26', '2011-07-03', '2011-07-10',
               '2011-07-17', '2011-07-24', '2011-07-31', '2011-08-07',
               '2011-08-14', '2011-08-21', '2011-08-28', '2011-09-04',
               '2011-09-11', '2011-09-18', '2011-09-25', '2011-10-02',
               '2011-10-09', '2011-10-16', '2011-10-23', '2011-10-30',
               '2011-11-06', '2011-11-13', '2011-11-20', '2011-11-27',
               '2011-12-04', '2011-12-11', '2011-12-18', '2011-12-25',
               '2012-01-01'],
              dtype='datetime64[ns]', freq='W-S

In [ ]:
from time import sleep
for cid,cname in charts.items():
    if daytype.get(cname) is None:
        continue
    dates = getDates(daytype, cname)
    
    ##savedir  = join(basedir, "data", "top40", "starters")
    ##savename = join(savedir, "{0}.p".format(cname.replace("/", " ")))
    ##starter  = getHTML(savename)
    #dates    = getDates(starter)
    
    for idts,datename in enumerate(dates):
        
        if idts > 2000:
            break
        url      = "https://top40-charts.com/chart.php?cid={0}&date={1}".format(cid, datename)
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,}
    
        savedir  = join(basedir, "data", "top40", cname.replace("/", " "))
        mkDir(savedir)

        savename = setFile(savedir, "{0}.p".format(datename))
        if isFile(savename):
            continue

        request=urllib.request.Request(url,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read() # The data u need

        print("URL ---> {0}".format(url))
        print(idts,'/',len(dates),"\tSaving {0}".format(savename))
        saveJoblib(data=data, filename=savename, compress=True)
        sleep(3)
        

#list(all_sundays(2001))

URL ---> https://top40-charts.com/chart.php?cid=36&date=2002-01-07
0 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-01-07.p
  --> This file is 35.4kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2002-01-14
1 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-01-14.p
  --> This file is 35.9kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2002-01-21
2 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-01-21.p
  --> This file is 35.9kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2002-02-02
3 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-02-02.p
  --> This file is 35.9kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2002-02-09
4 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-02-09.p
  --> This f

URL ---> https://top40-charts.com/chart.php?cid=36&date=2002-10-26
41 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-10-26.p
  --> This file is 36.5kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2002-11-02
42 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-11-02.p
  --> This file is 36.5kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2002-11-09
43 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-11-09.p
  --> This file is 36.7kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2002-11-16
44 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-11-16.p
  --> This file is 36.7kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2002-11-23
45 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2002-11-23.p
  --> T

URL ---> https://top40-charts.com/chart.php?cid=36&date=2003-08-09
82 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2003-08-09.p
  --> This file is 36.6kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2003-08-16
83 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2003-08-16.p
  --> This file is 36.6kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2003-08-23
84 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2003-08-23.p
  --> This file is 36.3kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2003-08-30
85 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2003-08-30.p
  --> This file is 36.6kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2003-09-06
86 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2003-09-06.p
  --> T

URL ---> https://top40-charts.com/chart.php?cid=36&date=2004-05-22
123 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2004-05-22.p
  --> This file is 36.8kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2004-05-29
124 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2004-05-29.p
  --> This file is 37.0kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2004-06-05
125 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2004-06-05.p
  --> This file is 37.2kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2004-06-12
126 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2004-06-12.p
  --> This file is 37.0kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2004-06-19
127 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2004-06-19.p
  

URL ---> https://top40-charts.com/chart.php?cid=36&date=2005-02-26
163 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2005-02-26.p
  --> This file is 37.0kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2005-03-05
164 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2005-03-05.p
  --> This file is 37.2kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2005-03-12
165 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2005-03-12.p
  --> This file is 36.9kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2005-03-19
166 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2005-03-19.p
  --> This file is 36.7kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2005-03-26
167 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2005-03-26.p
  

URL ---> https://top40-charts.com/chart.php?cid=36&date=2005-12-03
203 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2005-12-03.p
  --> This file is 37.3kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2005-12-10
204 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2005-12-10.p
  --> This file is 37.2kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2005-12-17
205 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2005-12-17.p
  --> This file is 37.9kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2005-12-24
206 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2005-12-24.p
  --> This file is 37.5kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2005-12-31
207 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2005-12-31.p
  

URL ---> https://top40-charts.com/chart.php?cid=36&date=2006-09-09
243 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2006-09-09.p
  --> This file is 24.8kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2006-09-16
244 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2006-09-16.p
  --> This file is 24.8kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2006-09-23
245 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2006-09-23.p
  --> This file is 24.8kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2006-09-30
246 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2006-09-30.p
  --> This file is 24.8kB.
URL ---> https://top40-charts.com/chart.php?cid=36&date=2006-10-07
247 / 926 	Saving /Users/tgadfort/Documents/code/charts/data/top40/Airplay World Official Top 100/2006-10-07.p
  

In [17]:
from time import sleep
for cid,cname in charts.items():
    if daytype.get(cname) is None:
        continue    
    dates = getDates(daytype, cname)
    
    for idts,datename in enumerate(dates):
        savedir  = join(basedir, "data", "top40", cname.replace("/", " "))
        savename = setFile(savedir, "{0}.p".format(datename))

        chartData = getHTML(savename)
        results = getChartData(chartData, debug=False)
        if len(results) == 0:
            print(cname,'\t\t',datename,'\t',len(results)," <<-------")
        else:
            print(cname,'\t\t',datename,'\t',len(results))

Airplay World Official Top 100 		 2002-01-07 	 100
Airplay World Official Top 100 		 2002-01-14 	 100
Airplay World Official Top 100 		 2002-01-21 	 100
Airplay World Official Top 100 		 2002-02-02 	 100
Airplay World Official Top 100 		 2002-02-09 	 100
Airplay World Official Top 100 		 2002-02-16 	 100
Airplay World Official Top 100 		 2002-02-23 	 100
Airplay World Official Top 100 		 2002-03-02 	 100
Airplay World Official Top 100 		 2002-03-09 	 100
Airplay World Official Top 100 		 2002-03-16 	 100
Airplay World Official Top 100 		 2002-03-23 	 100
Airplay World Official Top 100 		 2002-03-30 	 100
Airplay World Official Top 100 		 2002-04-06 	 100
Airplay World Official Top 100 		 2002-04-13 	 100
Airplay World Official Top 100 		 2002-04-20 	 100
Airplay World Official Top 100 		 2002-04-27 	 100
Airplay World Official Top 100 		 2002-05-04 	 100
Airplay World Official Top 100 		 2002-05-11 	 100
Airplay World Official Top 100 		 2002-05-18 	 100
Airplay World Official Top 100 

Airplay World Official Top 100 		 2005-02-12 	 100
Airplay World Official Top 100 		 2005-02-19 	 100
Airplay World Official Top 100 		 2005-02-26 	 100
Airplay World Official Top 100 		 2005-03-05 	 100
Airplay World Official Top 100 		 2005-03-12 	 100
Airplay World Official Top 100 		 2005-03-19 	 100
Airplay World Official Top 100 		 2005-03-26 	 100
Airplay World Official Top 100 		 2005-04-02 	 100
Airplay World Official Top 100 		 2005-04-09 	 100
Airplay World Official Top 100 		 2005-04-16 	 100
Airplay World Official Top 100 		 2005-04-23 	 100
Airplay World Official Top 100 		 2005-04-30 	 100
Airplay World Official Top 100 		 2005-05-07 	 100
Airplay World Official Top 100 		 2005-05-14 	 100
Airplay World Official Top 100 		 2005-05-21 	 100
Airplay World Official Top 100 		 2005-05-28 	 100
Airplay World Official Top 100 		 2005-06-04 	 100
Airplay World Official Top 100 		 2005-06-11 	 100
Airplay World Official Top 100 		 2005-06-18 	 100
Airplay World Official Top 100 

Airplay World Official Top 100 		 2009-02-21 	 0
Airplay World Official Top 100 		 2009-02-28 	 0
Airplay World Official Top 100 		 2009-03-07 	 0
Airplay World Official Top 100 		 2009-03-14 	 0
Airplay World Official Top 100 		 2009-03-21 	 0
Airplay World Official Top 100 		 2009-03-28 	 0
Airplay World Official Top 100 		 2009-04-04 	 0
Airplay World Official Top 100 		 2009-04-11 	 0
Airplay World Official Top 100 		 2009-04-18 	 0
Airplay World Official Top 100 		 2009-04-25 	 0
Airplay World Official Top 100 		 2009-05-02 	 0
Airplay World Official Top 100 		 2009-05-09 	 0
Airplay World Official Top 100 		 2009-05-16 	 0
Airplay World Official Top 100 		 2009-05-23 	 0
Airplay World Official Top 100 		 2009-05-30 	 0
Airplay World Official Top 100 		 2009-06-06 	 0
Airplay World Official Top 100 		 2009-06-13 	 0
Airplay World Official Top 100 		 2009-06-20 	 0
Airplay World Official Top 100 		 2009-06-27 	 0
Airplay World Official Top 100 		 2009-07-04 	 0
Airplay World Offici

Airplay World Official Top 100 		 2013-10-26 	 0
Airplay World Official Top 100 		 2013-11-02 	 0
Airplay World Official Top 100 		 2013-11-09 	 0
Airplay World Official Top 100 		 2013-11-16 	 0
Airplay World Official Top 100 		 2013-11-23 	 0
Airplay World Official Top 100 		 2013-11-30 	 0
Airplay World Official Top 100 		 2013-12-07 	 0
Airplay World Official Top 100 		 2013-12-14 	 0
Airplay World Official Top 100 		 2013-12-21 	 0
Airplay World Official Top 100 		 2013-12-28 	 0
Airplay World Official Top 100 		 2014-01-04 	 0
Airplay World Official Top 100 		 2014-01-11 	 0
Airplay World Official Top 100 		 2014-01-18 	 0
Airplay World Official Top 100 		 2014-01-25 	 0
Airplay World Official Top 100 		 2014-02-01 	 0
Airplay World Official Top 100 		 2014-02-08 	 0
Airplay World Official Top 100 		 2014-02-15 	 0
Airplay World Official Top 100 		 2014-02-22 	 0
Airplay World Official Top 100 		 2014-03-01 	 0
Airplay World Official Top 100 		 2014-03-08 	 0
Airplay World Offici

Airplay World Official Top 100 		 2019-01-05 	 0
Airplay World Official Top 100 		 2019-01-12 	 0
Airplay World Official Top 100 		 2019-01-19 	 0
Airplay World Official Top 100 		 2019-01-26 	 0
Airplay World Official Top 100 		 2019-02-02 	 0
Airplay World Official Top 100 		 2019-02-09 	 0
Airplay World Official Top 100 		 2019-02-16 	 0
Airplay World Official Top 100 		 2019-02-23 	 0
Airplay World Official Top 100 		 2019-03-02 	 0
Airplay World Official Top 100 		 2019-03-09 	 0
Airplay World Official Top 100 		 2019-03-16 	 0
Airplay World Official Top 100 		 2019-03-23 	 0
Airplay World Official Top 100 		 2019-03-30 	 0
Airplay World Official Top 100 		 2019-04-06 	 0
Airplay World Official Top 100 		 2019-04-13 	 0
Airplay World Official Top 100 		 2019-04-20 	 0
Airplay World Official Top 100 		 2019-04-27 	 0
Airplay World Official Top 100 		 2019-05-04 	 0
Airplay World Official Top 100 		 2019-05-11 	 0
Airplay World Official Top 100 		 2019-05-18 	 0
Airplay World Offici

# Process Chart Data

In [49]:
savedir  = join(basedir, "data", "top40")
dirnames = [x for x in findDirs(savedir) if x.endswith("starters") == False]
for dirname in dirnames:
    files = findExt(dirname, ext="*")
    for ifile in files:
        src = ifile
        dst = "{0}.p".format(ifile)
        moveFile(src, dst)


In [11]:
savedir  = join(basedir, "data", "top40")
dirnames = [x for x in findDirs(savedir) if x.endswith("starters") == False]
for dirname in dirnames:
    files = findExt(dirname, ext="*.p")
    for ifile in files:
        chartData = getHTML(ifile)
        break

In [12]:
chartData

<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#"><head>
<!-- logpage chart 139  --><!-- Hotjar Tracking Code for top40-charts.com -->
<script>
    (function(h,o,t,j,a,r){
        h.hj=h.hj||function(){(h.hj.q=h.hj.q||[]).push(arguments)};
        h._hjSettings={hjid:1094028,hjsv:6};
        a=o.getElementsByTagName('head')[0];
        r=o.createElement('script');r.async=1;
        r.src=t+h._hjSettings.hjid+j+h._hjSettings.hjsv;
        a.appendChild(r);
    })(window,document,'https://static.hotjar.com/c/hotjar-','.js?sv=');
</script><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>World Country Top 20 Singles @  Top40-Charts.com - New Songs &amp; Videos from 49 Top 20 &amp; Top 40 Music Charts from 30 Countries</title>
<meta content="World Country Top 20 Singles on Top40 Charts. Top40-Charts.com provides music charts from all over the world, like US / UK Albums and Singles, Bilboard Chart, Dance charts and more. Read the latest music news on rock, pop, 

In [141]:
chartData = getHTML("/Users/tgadfort/Documents/code/charts/data/top40/test.html")

In [13]:
def getChartData(chartData):
    artists = []
    titles = []
    
    trs = chartData.findAll("tr", {"class": "latc_song"})    
    for xs in [x.findAll("a", {"title": "View song details"}) for x in trs]:
        titles.append([x.text for x in xs if x.find("img") == None][0])

    for xs in [x.findAll("a", {"style": "text-decoration: none; "}) for x in trs]:
        artists.append(xs[0].text)
    
    retval = dict(zip(artists, titles))
    return retval

In [17]:

len(chartData.findAll('table'))

19

In [18]:
from searchUtils import findExt

In [21]:
files = findExt("data/top40/World Singles Official Top 100/", ext=".p")

In [27]:
for ifile in files[:5]:
    print(ifile)
    chartData = getHTML(ifile)
    
    for it,x in enumerate(chartData.findAll("table")):
        trs = x.findAll("tr", {"class": "latc_song"})
        for itr,tr in enumerate(trs):
            tds = tr.findAll("td")
            for itd,td in enumerate(tds):
                hrefs = td.findAll("a")
                for ihref,href in enumerate(hrefs):
                    print(it,'\t',itr,'\t',itd,'\t',ihref,'\t',href.text)

data/top40/World Singles Official Top 100/2002-01-07.p
data/top40/World Singles Official Top 100/2002-01-14.p
data/top40/World Singles Official Top 100/2002-01-21.p
data/top40/World Singles Official Top 100/2002-02-02.p
data/top40/World Singles Official Top 100/2002-02-09.p


In [30]:
for it,x in enumerate(chartData.findAll("table")):
    print(it,len(x))

0 2
1 4
2 3
3 3
4 1
5 2
6 1
7 2
8 15
9 2
10 1
11 5
12 1
13 1
14 1
15 3
16 3
17 1
18 1


In [31]:
chartData.findAll("table")[8]

<table align="right" border="0" cellpadding="0" cellspacing="0" width="400">
<tr><td class="small" width="230">
</td>
<td class="small" width="90"><img border="0" height="1" src="https://maxcdn.top40-charts.com/images/spacer.gif" width="90"/></td>
<td align="center" bgcolor="F17D38" class="bigger" nowrap="" width="150"><font color="f8fafa"><b> Charts </b></font></td></tr>
<tr height="1"><td bgcolor="F17D38" colspan="3"><img border="0" height="1" src="https://maxcdn.top40-charts.com/images/spacer.gif" width="200"/></td></tr>
<!-- Vibrant Media IntelliTXT id -->
<div id="intelliTxt"></div>
<!-- filename /home/top40-ch/www/html/charts/chart-35-2001-07-29.html proceed 0 status 4 load  timediff 0.00019675925925926 -->
<!-- try to include -->
<tr><td align="left" bgcolor="F17D38" nowrap=""><table border="0" cellpadding="2" cellspacing="0" width="100%">
<tr><td nowrap=""><form action="chart.php" style="margin: 0px;"><select name="cid"><option value="36">Airplay World Official Top 100</option>

In [36]:
files = findExt("data/top40/World Singles Official Top 100/", ext=".p")

In [38]:
files[0]

'data/top40/World Singles Official Top 100/2002-01-07.p'

In [39]:
getHTML(files[0])

<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#"><head>
<!-- logpage chart 133  --><!-- Hotjar Tracking Code for top40-charts.com -->
<script>
    (function(h,o,t,j,a,r){
        h.hj=h.hj||function(){(h.hj.q=h.hj.q||[]).push(arguments)};
        h._hjSettings={hjid:1094028,hjsv:6};
        a=o.getElementsByTagName('head')[0];
        r=o.createElement('script');r.async=1;
        r.src=t+h._hjSettings.hjid+j+h._hjSettings.hjsv;
        a.appendChild(r);
    })(window,document,'https://static.hotjar.com/c/hotjar-','.js?sv=');
</script><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>World Singles Official Top 100 @  Top40-Charts.com - New Songs &amp; Videos from 49 Top 20 &amp; Top 40 Music Charts from 30 Countries</title>
<meta content="World Singles Official Top 100 on Top40 Charts. Top40-Charts.com provides music charts from all over the world, like US / UK Albums and Singles, Bilboard Chart, Dance charts and more. Read the latest music news on rock, p

In [48]:
def all_sundays(year):
# January 1st of the given year
       dt = date(year, 1, 1)
# First Sunday of the given year       
       dt += timedelta(days = 6 - dt.weekday())  
       while dt.year == year:
            yield dt
            dt += timedelta(days = 7)
            
for s in all_sundays(2020):
    m = s.month
    d = s.day
    y = s.year
    print(s.strftime("%d-%m-%Y"))
    print(type(s))

05-01-2020
<class 'datetime.date'>
12-01-2020
<class 'datetime.date'>
19-01-2020
<class 'datetime.date'>
26-01-2020
<class 'datetime.date'>
02-02-2020
<class 'datetime.date'>
09-02-2020
<class 'datetime.date'>
16-02-2020
<class 'datetime.date'>
23-02-2020
<class 'datetime.date'>
01-03-2020
<class 'datetime.date'>
08-03-2020
<class 'datetime.date'>
15-03-2020
<class 'datetime.date'>
22-03-2020
<class 'datetime.date'>
29-03-2020
<class 'datetime.date'>
05-04-2020
<class 'datetime.date'>
12-04-2020
<class 'datetime.date'>
19-04-2020
<class 'datetime.date'>
26-04-2020
<class 'datetime.date'>
03-05-2020
<class 'datetime.date'>
10-05-2020
<class 'datetime.date'>
17-05-2020
<class 'datetime.date'>
24-05-2020
<class 'datetime.date'>
31-05-2020
<class 'datetime.date'>
07-06-2020
<class 'datetime.date'>
14-06-2020
<class 'datetime.date'>
21-06-2020
<class 'datetime.date'>
28-06-2020
<class 'datetime.date'>
05-07-2020
<class 'datetime.date'>
12-07-2020
<class 'datetime.date'>
19-07-2020
<class 'd

In [57]:
d = "2002-01-07"

In [60]:
files = findExt("data/top40/old/World Singles Official Top 100/", ext=".p")

In [63]:
bs = getHTML(files[0])

In [65]:
bs.find("Iglesias")

In [66]:
url = "https://top40-charts.com/chart.php?cid=35&date=2002-01-07"

In [68]:
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,}

savename = setFile(savedir, "mytest.p")

request=urllib.request.Request(url,None,headers) #The assembled request
response = urllib.request.urlopen(request)
data = response.read() # The data u need

print(idts,'/',len(dates),"\tSaving {0}".format(savename))
saveJoblib(data=data, filename=savename, compress=True)

302 / 303 	Saving /Users/tgadfort/Documents/code/charts/data/top40/starters/mytest.p
  --> This file is 34.7kB.


In [75]:
str(data).find("Enya")

118349